In [3]:
# imports
import pandas as pd
import numpy as np
import requests
import json
import time

In [7]:
# Read in credentials
with open("../credentials.json") as f:
    credentials = json.load(f)
    yelp_api_key = credentials["yelp_api_key"]
    yelp_client_id = credentials["yelp_client_id"]
    google_maps_key = credentials["google_maps_key"]

In [93]:
from tqdm import tqdm

# Can set the total results
total_results = 2000

# Setting offsets to move to the next set of results
offsets = np.arange(0, total_results, 50)

# Empty list to fill
all_data = []

# Collecting data
for offset in tqdm(offsets):
    # API Call
    url = "https://api.yelp.com/v3/businesses/search?location=Washington%2C%20DC&sort_by=review_count&limit=50&offset=" + str(offset)

    # Authentication
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {yelp_api_key}"
    }

    # Calling API
    response = requests.get(url, headers=headers)

    # Parsing and adding data
    parsed_data = json.loads(response.text)
    all_data.extend(parsed_data.get('businesses', []))  # Append businesses to the list

    # Pausing
    time.sleep(0.6)


100%|██████████| 40/40 [00:49<00:00,  1.24s/it]


In [94]:
ids = []
names = []
review_counts = []
ratings = []
main_categories = []
categories_list = []
urls = []
prices = []


for item in all_data:
    # Collecting the values we care about
    id = item["id"]
    name = item['name']
    review_count = item['review_count']
    rating = item['rating']
    categories = item['categories']
    url = item['url']

    # Getting Price if they
    try:
        price = item['price']
    except Exception as e:
        price = "Not Listed"

    # Extracting Category Titles
    category_titles = [category['title'] for category in categories]

    # Adding to lists
    ids.append(id)
    names.append(name)
    review_counts.append(review_count)
    ratings.append(rating)
    main_categories.append(category_titles[0])
    categories_list.append(category_titles)
    urls.append(url)
    prices.append(price)

# Creating df
df = pd.DataFrame({'id':ids,
"name": names,
"yelp_review_count": review_counts,
"yelp_rating": ratings,
"price":prices,
"main_category":main_categories,
"categories":categories_list,
"url":urls})

df.head()


,id,name,review_count,rating,price,main_category,categories,url
0,VA8aPObRynlwR1TGzbzraQ,Founding Farmers - Washington,16415,4.0,$$,American (Traditional),"[American (Traditional), Coffee & Tea, Breakfa...",https://www.yelp.com/biz/founding-farmers-wash...
1,iyBbcXtQSBfiwFQZwVBNaQ,Old Ebbitt Grill,10324,4.0,$$,Bars,"[Bars, American (Traditional), Breakfast & Bru...",https://www.yelp.com/biz/old-ebbitt-grill-wash...
2,GBkFa8TJwkaUJsJXXGkTTg,Zaytinya,5650,4.0,$$$,Greek,"[Greek, Turkish, Lebanese]",https://www.yelp.com/biz/zaytinya-washington?a...
3,Es64Je53efmpWh-BXHjguQ,Baked & Wired,5068,4.5,$$,Bakeries,"[Bakeries, Coffee & Tea, Cupcakes]",https://www.yelp.com/biz/baked-and-wired-washi...
4,j9qYRR8HCXm_GEnetijOGA,Le Diplomate,4928,4.5,$$$,Brasseries,"[Brasseries, French, Breakfast & Brunch]",https://www.yelp.com/biz/le-diplomate-washingt...


In [116]:
# Setting up Google Maps API
import googlemaps
gmaps = googlemaps.Client(key=google_maps_key)

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'formatted_address': '701 9th St NW, Washington, DC 20001, United States',
   'geometry': {'location': {'lat': 38.8987321, 'lng': -77.02368299999999},
    'viewport': {'northeast': {'lat': 38.90000007989271,
      'lng': -77.02235122010728},
     'southwest': {'lat': 38.89730042010727, 'lng': -77.02505087989273}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
   'icon_background_color': '#FF9E67',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
   'name': 'Zaytinya by José Andrés',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 1200,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/118429381734607593042">A Google User</a>'],
     'photo_reference': 'ATJ83zg03RB4pZWQdE8Fjq5vSnN-b0tD0kLFKd6TNXswVdbDGEQsheVRdLCBH3VdWoiTPBjPCYeBx4IFp7TQS293SgB8qM5dYK-BjMvCMB0v6H822jTOIbia7VxW

In [ ]:
google_ratings = []
google_review_count = []

# Gathering google maps results
from tqdm import tqdm
for place in tqdm(df['name']):
    # Trying to find restaurants under the same results
    # This is inexact; would like a better version using Longitude and Lattitude data for precision
    try:
        results = gmaps.places(query=place + " DC")['results'][0]

        # Gathering Ratings
        # All other data we will use yelp
        rating = results['rating']
        review_count = results['user_ratings_total']

        # Add to list
        google_ratings.append(rating)
        google_review_count.append(review_count)
    
    except IndexError:
        # Handle no results found
        google_ratings.append(None)
        google_review_count.append(None)
        
    except Exception as e:
        print(f"An error occurred for place '{place}': {str(e)}")
        google_ratings.append(None)
        google_review_count.append(None)
    
    time.sleep(2)



# Add to Dataframe
df["google_rating"] = google_ratings
df["google_review_count"] = google_review_count


df.head()

In [146]:
# Add to Dataframe
df["google_rating"] = google_ratings
df["google_review_count"] = google_review_count


df.head()

,id,name,yelp_review_count,yelp_rating,price,main_category,categories,url,google_rating,google_review_count
0,VA8aPObRynlwR1TGzbzraQ,Founding Farmers - Washington,16415,4.0,$$,American (Traditional),"[American (Traditional), Coffee & Tea, Breakfa...",https://www.yelp.com/biz/founding-farmers-wash...,4.4,14762.0
1,iyBbcXtQSBfiwFQZwVBNaQ,Old Ebbitt Grill,10324,4.0,$$,Bars,"[Bars, American (Traditional), Breakfast & Bru...",https://www.yelp.com/biz/old-ebbitt-grill-wash...,4.6,15505.0
2,GBkFa8TJwkaUJsJXXGkTTg,Zaytinya,5650,4.0,$$$,Greek,"[Greek, Turkish, Lebanese]",https://www.yelp.com/biz/zaytinya-washington?a...,4.5,6662.0
3,Es64Je53efmpWh-BXHjguQ,Baked & Wired,5068,4.5,$$,Bakeries,"[Bakeries, Coffee & Tea, Cupcakes]",https://www.yelp.com/biz/baked-and-wired-washi...,4.6,3240.0
4,j9qYRR8HCXm_GEnetijOGA,Le Diplomate,4928,4.5,$$$,Brasseries,"[Brasseries, French, Breakfast & Brunch]",https://www.yelp.com/biz/le-diplomate-washingt...,4.6,5926.0


In [144]:
df.to_csv('../data/dc_reviews.csv')

In [27]:
from ast import literal_eval
df = pd.read_csv('../data/dc_reviews.csv', index_col=0, converters={'categories': literal_eval})
df.head()

,id,name,review_count,rating,price,main_category,categories,url,google_rating,google_review_count
0,VA8aPObRynlwR1TGzbzraQ,Founding Farmers - Washington,16415,4.0,$$,American (Traditional),"[American (Traditional), Coffee & Tea, Breakfa...",https://www.yelp.com/biz/founding-farmers-wash...,4.4,14762.0
1,iyBbcXtQSBfiwFQZwVBNaQ,Old Ebbitt Grill,10324,4.0,$$,Bars,"[Bars, American (Traditional), Breakfast & Bru...",https://www.yelp.com/biz/old-ebbitt-grill-wash...,4.6,15505.0
2,GBkFa8TJwkaUJsJXXGkTTg,Zaytinya,5650,4.0,$$$,Greek,"[Greek, Turkish, Lebanese]",https://www.yelp.com/biz/zaytinya-washington?a...,4.5,6662.0
3,Es64Je53efmpWh-BXHjguQ,Baked & Wired,5068,4.5,$$,Bakeries,"[Bakeries, Coffee & Tea, Cupcakes]",https://www.yelp.com/biz/baked-and-wired-washi...,4.6,3240.0
4,j9qYRR8HCXm_GEnetijOGA,Le Diplomate,4928,4.5,$$$,Brasseries,"[Brasseries, French, Breakfast & Brunch]",https://www.yelp.com/biz/le-diplomate-washingt...,4.6,5926.0


In [28]:
avg_ratings = []
total_reviews_list = []

# Fixing my old column names
if 'review_count' in df.columns:
    df = df.rename(columns={'rating': 'yelp_rating', 'review_count': 'yelp_review_count'})

# Combine American (Traditional) and American (New) main categories to American
df['main_category'] = df['main_category'].replace(['American (New)', 'American (Traditional)'], 'American')

# Combine American (Traditional) and American (New) categories to American
df['categories'] = df['categories'].apply(lambda x: [category.replace('American (New)', 'American').replace('American (Traditional)', 'American') for category in x])

# Merging the review scores
for id in df["id"]:

    # Getting Yelp data
    yelp_rating = df[df['id'] == id]['yelp_rating'].item()
    yelp_rc = df[df['id'] == id]['yelp_review_count'].item()
    yelp_total = yelp_rating * yelp_rc

    # Google data
    google_rating = df[df['id'] == id]['google_rating'].item()
    google_rc = df[df['id'] == id]['google_review_count'].item()
    google_total = google_rating * google_rc  

    # Finding the total reviews and overall average
    total_reviews = google_rc + yelp_rc
    avg_rating = round((google_total + yelp_total) / total_reviews, 4)

    # Add to lists
    avg_ratings.append(avg_rating)
    total_reviews_list.append(total_reviews)

# Add to DataFrame
df['average_rating'] = avg_ratings
df['total_reviews'] = total_reviews_list
    
# Finding the Top 10 Categories
flattened_categories = [category for sublist in df['categories'] for category in sublist]

# Count the occurrences of each category
from collections import Counter
category_counts = Counter(flattened_categories)

# Get the top 10 categories
top_10_categories = dict(category_counts.most_common(10))
top_10_categories_list = list(top_10_categories.keys())

# Putting a restaurants Main Category as what is in the top 10 or Other
for i in range(len(df['main_category'])):
    # Get main category
    main_category = df['main_category'][i]

    # Check if it is already in the top 10
    if main_category not in list(top_10_categories.keys()):
        # Check if any category_list item is in the top 10 categories
        for category in df['categories'][i]:
            if category in top_10_categories:
                # Replace 'main_category' with the category from 'category_list'
                df.at[i, 'main_category'] = category
                break  # Stop after finding a replacement
            else:
                df.at[i, 'main_category'] = "Other"

In [29]:
df.head()

,id,name,yelp_review_count,yelp_rating,price,main_category,categories,url,google_rating,google_review_count,average_rating,total_reviews
0,VA8aPObRynlwR1TGzbzraQ,Founding Farmers - Washington,16415,4.0,$$,American,"[American, Coffee & Tea, Breakfast & Brunch]",https://www.yelp.com/biz/founding-farmers-wash...,4.4,14762.0,4.1894,31177.0
1,iyBbcXtQSBfiwFQZwVBNaQ,Old Ebbitt Grill,10324,4.0,$$,Bars,"[Bars, American, Breakfast & Brunch]",https://www.yelp.com/biz/old-ebbitt-grill-wash...,4.6,15505.0,4.3602,25829.0
2,GBkFa8TJwkaUJsJXXGkTTg,Zaytinya,5650,4.0,$$$,Other,"[Greek, Turkish, Lebanese]",https://www.yelp.com/biz/zaytinya-washington?a...,4.5,6662.0,4.2705,12312.0
3,Es64Je53efmpWh-BXHjguQ,Baked & Wired,5068,4.5,$$,Coffee & Tea,"[Bakeries, Coffee & Tea, Cupcakes]",https://www.yelp.com/biz/baked-and-wired-washi...,4.6,3240.0,4.5390,8308.0
4,j9qYRR8HCXm_GEnetijOGA,Le Diplomate,4928,4.5,$$$,Breakfast & Brunch,"[Brasseries, French, Breakfast & Brunch]",https://www.yelp.com/biz/le-diplomate-washingt...,4.6,5926.0,4.5546,10854.0


In [32]:
new_df = df[df["total_reviews"] >= 100]

# Remove museums, monuments, memorials, and libraries
new_df = new_df[~new_df['name'].str.contains("museum", case=False)]
new_df = new_df[~new_df['name'].str.contains("national", case=False)]
new_df = new_df[~new_df['name'].str.contains("monument", case=False)]
new_df = new_df[~new_df['name'].str.contains("memorial", case=False)]
new_df = new_df[~new_df['name'].str.contains("united states", case=False)]
new_df = new_df[~new_df['name'].str.contains("library", case=False)]

# Normalize the average rating
new_df['normalized_rating'] = (new_df['average_rating'] - new_df['average_rating'].min()) / (new_df['average_rating'].max() - new_df['average_rating'].min()) * 100

# Rounding the normalized rating
new_df['rounded_normalized_rating'] = round(new_df['normalized_rating'], 2)

# Normalize the total reviews
new_df['normalized_total_reviews'] = (new_df['total_reviews'] - new_df['total_reviews'].min()) / (new_df['total_reviews'].max() - new_df['total_reviews'].min()) * 100

# Rounding the normalized total reviews
new_df['rounded_normalized_total_reviews'] = round(new_df['normalized_total_reviews'], 2)

# Save to CSV
new_df.to_csv('../data/dc_reviews_cleaned.csv', index=False)

# Display
new_df.head()

,id,name,yelp_review_count,yelp_rating,price,main_category,categories,url,google_rating,google_review_count,average_rating,total_reviews,normalized_rating,rounded_normalized_rating,normalized_total_reviews,rounded_normalized_total_reviews
0,VA8aPObRynlwR1TGzbzraQ,Founding Farmers - Washington,16415,4.0,$$,American,"[American, Coffee & Tea, Breakfast & Brunch]",https://www.yelp.com/biz/founding-farmers-wash...,4.4,14762.0,4.1894,31177.0,67.408784,67.41,100.000000,100.00
1,iyBbcXtQSBfiwFQZwVBNaQ,Old Ebbitt Grill,10324,4.0,$$,Bars,"[Bars, American, Breakfast & Brunch]",https://www.yelp.com/biz/old-ebbitt-grill-wash...,4.6,15505.0,4.3602,25829.0,74.792495,74.79,82.616610,82.62
2,GBkFa8TJwkaUJsJXXGkTTg,Zaytinya,5650,4.0,$$$,Other,"[Greek, Turkish, Lebanese]",https://www.yelp.com/biz/zaytinya-washington?a...,4.5,6662.0,4.2705,12312.0,70.914750,70.91,38.680319,38.68
3,Es64Je53efmpWh-BXHjguQ,Baked & Wired,5068,4.5,$$,Coffee & Tea,"[Bakeries, Coffee & Tea, Cupcakes]",https://www.yelp.com/biz/baked-and-wired-washi...,4.6,3240.0,4.5390,8308.0,82.522047,82.52,25.665529,25.67
4,j9qYRR8HCXm_GEnetijOGA,Le Diplomate,4928,4.5,$$$,Breakfast & Brunch,"[Brasseries, French, Breakfast & Brunch]",https://www.yelp.com/biz/le-diplomate-washingt...,4.6,5926.0,4.5546,10854.0,83.196438,83.20,33.941167,33.94


In [31]:
import plotly.express as px
# Find averages for quadrant lines
x_avg = new_df['normalized_rating'].mean()
y_avg = new_df['normalized_total_reviews'].mean()

# Create Scatter
fig = px.scatter(
    new_df,
    x="normalized_rating",
    y="normalized_total_reviews",
    color="main_category",
    hover_name="name",
    custom_data=["name", "price", "average_rating", "rounded_normalized_rating", "total_reviews"],
    title="DC Restaurant Grid",
    labels={"main_category": "Category"},
    # text='name'
)

# Update the hover template to include only 'name' and 'price'
fig.update_traces(
    hovertemplate="<b>%{hovertext}</b><br>Name: %{customdata[0]}<br>Price: %{customdata[1]}<br>Rating: %{customdata[2]}<br>Normalized Rating: %{customdata[3]}<br>Review Count: %{customdata[4]:,}"
)

# Add quadrant Lines: https://stackoverflow.com/questions/73278985/how-can-i-add-annotations-outside-a-plotly-quandrant-chart
fig.add_vline(x=x_avg, line_width=1, opacity=0.5)
fig.add_hline(y=y_avg, line_width=1, opacity=0.5)

# Add quadrant text
fig.add_annotation(x=x_avg + 25, y=y_avg + 50, 
            text="Deservedly Popular",
            showarrow=False)
fig.add_annotation(x=x_avg + 25, y=y_avg - 10, 
            text="Hidden Gems",
            showarrow=False)
fig.add_annotation(x=x_avg - 25, y=y_avg + 50,
            text="Overrated",
            showarrow=False)
fig.add_annotation(x=x_avg - 25, y=y_avg - 10, 
            text="Not Worth It",
            showarrow=False)

category_options = [
    {
        "label": "All Restaurants",
        "method": "update",
        "args": [
            {"visible": [True] * len(new_df)},
            {"title.text": "DC Restaurant Grid - All Restaurants"},
        ],
    }
]

for category in new_df["main_category"].unique():
    category_bools = new_df["main_category"] == category
    category_options += [
        {
            "label": category,
            "method": "update",
            "args": [
                {"visible": category_bools},
                {"title.text": f"DC Restaurant Grid - Category: {category}"},
            ],
        }
    ]

# Add the dropdown menu
fig.update_layout(
    updatemenus=[
        {
            "buttons": category_options,
            "direction": "down",
            "showactive": True,
            "x": 1.05,
            "xanchor": "left",
            "y": 0.45,
            "yanchor": "top",
        }
    ]
)

# Clean
fig.update_layout(
    template="plotly_white",
    xaxis_title="Normalized Rating",
    yaxis_title="Popularity Score (Normalized Review Count)",
    height=1000,  # Set the height
    width=1000,  # Set the width
)

# fig.update_traces(textposition='top center')

# Display
fig.show()


American
0     True
1    False
2    False
3    False
4    False
5     True
6    False
7    False
8    False
9    False
Name: main_category, dtype: bool 0              American
1                  Bars
2                 Other
3          Coffee & Tea
4    Breakfast & Brunch
5              American
6               Italian
7                 Other
8            Sandwiches
9                 Other
Name: main_category, dtype: object
Bars
0    False
1     True
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
Name: main_category, dtype: bool 0              American
1                  Bars
2                 Other
3          Coffee & Tea
4    Breakfast & Brunch
5              American
6               Italian
7                 Other
8            Sandwiches
9                 Other
Name: main_category, dtype: object
Other
0    False
1    False
2     True
3    False
4    False
5    False
6    False
7     True
8    False
9     True
Name: main_category, dtype: bool 0

/var/folders/hg/dd3yfd8j7vx8qtmvm42400j80000gn/T/ipykernel_88423/3440869652.py:56: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.

